In [1]:
import os
import pandas as pd
from flask import Flask,render_template,redirect,request
from werkzeug.utils import secure_filename

from resume_screening import resparser, match

c:\Users\Acer\anaconda3\envs\resume-parser\lib\site-packages\spacy\util.py:275: UserWarning: [W031] Model 'en_training' (0.0.0) requires spaCy v2.2 and is incompatible with the current spaCy version (2.3.5). This may lead to unexpected results or runtime errors. To resolve this, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


In [2]:
import nltk
#nltk.download('punkt')
#nltk.download('averaged_perceptron_tagger')
#nltk.download('universal_tagset')
#nltk.download('maxent_ne_chunker')
#nltk.download('stopwords')
#nltk.download('wordnet')
#nltk.download('brown')

from nltk.corpus import stopwords
stopw  = set(stopwords.words('english'))

In [3]:
job = pd.read_csv('indeed_data.csv')
job['test'] = job['description'].apply(lambda x: ' '.join([word for word in str(x).split() if len(word)>2 and word not in (stopw)]))
df = job.drop_duplicates(subset='test').reset_index(drop=True)
df['clean'] = df['test'].apply(match.preprocessing)
jobdesc = (df['clean'].values.astype('U'))

In [23]:
app=Flask(__name__)

os.makedirs(os.path.join(app.instance_path, 'resume_files'), exist_ok=True)

@app.route('/')
def index():
    return render_template("index.html")


@app.route('/about')
def about():
    return render_template('about.html')

@app.route("/home")
def home():
    return redirect('/')


@app.route('/submit',methods=['POST'])
def submit_data():
    if request.method == 'POST':
        
        f=request.files['userfile']
        f.save(os.path.join(app.instance_path, 'resume_files', secure_filename(f.filename)))
        skills = resparser.skill('resume_files/{}'.format(f.filename))
        skills.append(match.preprocessing(skills[0]))
        del skills[0]

        count_matrix = match.vectorizing(skills[0], jobdesc)
        matchPercentage = match.coSim(count_matrix)
        matchPercentage = pd.DataFrame(matchPercentage, columns=['Skills Match'])

        #Job Vacancy Recommendations
        result_cosine = df[['title','company','link']]
        result_cosine = result_cosine.join(matchPercentage)
        result_cosine = result_cosine[['title','company','Skills Match','link']]
        result_cosine.columns = ['Job Title','Company','Skills Match','Link']
        result_cosine = result_cosine.sort_values('Skills Match', ascending=False).reset_index(drop=True).head(20)

    return render_template('index.html', column_names=result_cosine.columns.values, row_data=list(result_cosine.values.tolist()),
                           link_column="Link", zip=zip)


if __name__ =="__main__":
    
    
    app.run()

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000 (Press CTRL+C to quit)
127.0.0.1 - - [13/Jun/2022 10:53:01] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [13/Jun/2022 10:53:01] "GET /static/img/logo.png HTTP/1.1" 304 -
127.0.0.1 - - [13/Jun/2022 10:53:01] "GET /static/styles/style.css HTTP/1.1" 200 -
127.0.0.1 - - [13/Jun/2022 10:53:01] "GET /static/java/button.js HTTP/1.1" 304 -
127.0.0.1 - - [13/Jun/2022 10:53:01] "GET /static/img/telegram.png HTTP/1.1" 304 -
127.0.0.1 - - [13/Jun/2022 10:53:01] "GET /static/img/file-upload.png HTTP/1.1" 304 -
127.0.0.1 - - [13/Jun/2022 10:53:33] "POST /submit HTTP/1.1" 200 -
127.0.0.1 - - [13/Jun/2022 10:53:33] "GET /static/styles/style.css HTTP/1.1" 304 -
127.0.0.1 - - [13/Jun/2022 10:53:33] "GET /static/img/logo.png HTTP/1.1" 304 -
127.0.0.1 - - [13/Jun/2022 10:53:33] "GET /static/java/button.js HTTP/1.1" 304 -
127.0.0.1 - - [13/Jun/2022 10:53:33] "GET /static/img/file-upload.png HTTP/1.1" 304 -
127.0.0.1 - - [13/Jun/2022 10:53:33] "GET /static/img/telegram.png HTTP/1